In [770]:
from collections import defaultdict, Counter

god_rmse = defaultdict(list)

In [760]:
#from mangaki.utils.xals import MangakiXALS
from mangaki.utils.als import MangakiALS
from mangaki.utils.svd import MangakiSVD

In [761]:
from mangaki.utils.algo import get_dataset_backup
from mangaki.utils.data import Dataset
from mangaki.utils.values import rating_values

dataset = Dataset()#get_dataset_backup('als')
dataset.load_csv('balse/ratings.csv', convert=lambda x: rating_values[x])

In [762]:
from mangaki.utils.balse import MangakiBALSE
from scipy.sparse import load_npz

T = load_npz('../data/balse/tag-matrix.npz').toarray()

In [765]:
from sklearn.model_selection import ShuffleSplit

NB_SPLIT = 5
k_fold = ShuffleSplit(n_splits=NB_SPLIT)
SETS = list(k_fold.split(dataset.anonymized.X))

In [903]:
GOD_I = 4

In [904]:
# START
i_train, i_test = SETS[GOD_I]

In [905]:
NOTVAL = round(0.7 * len(i_train))
X_train = dataset.anonymized.X[i_train[:NOTVAL]]
y_train = dataset.anonymized.y[i_train[:NOTVAL]]
X_val = dataset.anonymized.X[i_train[NOTVAL:]]
y_val = dataset.anonymized.y[i_train[NOTVAL:]]
X_test = dataset.anonymized.X[i_test]
y_test = dataset.anonymized.y[i_test]
nb_train_rated = Counter(X_train[:, 1])

In [906]:
als = run_als(X_train, X_test, y_train, y_test)

Computing M: (2079 × 9979)
Chrono: fill and center matrix [0q, 1352ms]
Shapes (2079, 10) (10, 9979)
Chrono: factor matrix [0q, 5778ms]


In [907]:
lasso = run_lasso(X_train, X_test, y_train, y_test)

/Users/jilljenn/code/mangaki/venv/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1800
1900
2000


In [908]:
nb_rated = Counter(dataset.anonymized.X[:, 1])

In [909]:
def run_als(X_train, X_test, y_train, y_test):
    als = MangakiALS(10, LAMBDA=0.1)
    als.set_parameters(dataset.anonymized.nb_users, dataset.anonymized.nb_works)
    als.fit(X_train, y_train)
    #als.all_errors(X_train, X_test, y_train, y_test)
    return als

In [910]:
#y_pred = als.predict(X_tmp_test)
#y_new_pred = lasso.predict(X_tmp_test)

In [911]:
i_test_ordered = sorted(i_test, key=lambda i: nb_train_rated[dataset.anonymized.X[i][1]], reverse=True)

In [912]:
X_tmp_test = dataset.anonymized.X[i_test_ordered]
y_tmp_test = dataset.anonymized.y[i_test_ordered]

In [913]:
from mangaki.utils.lasso import MangakiLASSO

def run_lasso(X_train, X_test, y_train, y_test):
    lasso = MangakiLASSO(10, LAMBDA=0.1)
    lasso.load_tags(T)
    lasso.set_parameters(dataset.anonymized.nb_users, dataset.anonymized.nb_works)
    lasso.fit(X_train, y_train)
    #lasso.all_errors(X_train, X_test, y_train, y_test)
    return lasso

In [914]:
#y_train_als = als.predict(X_train)
#y_train_lasso = lasso.predict(X_train)

In [915]:
nb_r = list(map(lambda x: nb_train_rated[x[1]], X_val))

In [916]:
y_val_als = als.predict(X_val)
y_val_lasso = lasso.predict(X_val)

1138 not
1822 not
1765 not
703 not
1001 not
1392 not
1097 not
1495 not
1782 not
265 not
878 not
702 not
1441 not
588 not
1508 not
1040 not
160 not
1951 not
262 not
1495 not
159 not
990 not
1782 not
1887 not
2004 not
881 not
954 not
1161 not
878 not


In [917]:
y_als = als.predict(X_tmp_test)
y_lasso = lasso.predict(X_tmp_test)

890 not
104 not
1263 not
434 not
1177 not
2025 not
1822 not
1822 not
1951 not
1584 not
1700 not


In [918]:
import numpy as np

X = np.column_stack((nb_r, y_val_als, y_val_lasso))
y = y_val

In [919]:
import tensorflow as tf

In [920]:
beta = tf.Variable(tf.random_normal([1]), name='beta')
gamma = tf.Variable(tf.random_normal([1]), name='gamma')

In [921]:
e1 = np.array([1, 0, 0])
e2 = np.array([0, 1, 0])
e3 = np.array([0, 0, 1])
pred = tf.sigmoid(beta*(X.dot(e1) - gamma)) * X.dot(e2) + (1 - tf.sigmoid(beta*(X.dot(e1) - gamma))) * X.dot(e3)
loss = tf.reduce_mean(tf.square(y - pred)) ** 0.5
reg_loss = loss

global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 0.1
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           10, 0.9965402628278678, staircase=True)
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_step = optimizer.minimize(reg_loss, var_list=[beta, gamma], global_step=global_step)

init_op = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init_op)

In [922]:
for i in range(2000):
    sess.run(train_step)
    if i % 100 == 0:
        print('loss', sess.run(reg_loss), 'beta', beta.eval(), 'gamma', gamma.eval(), learning_rate.eval())

loss 1.18133 beta [ 0.13050348] gamma [-1.08007371] 0.1
loss 1.17722 beta [ 0.02510466] gamma [-1.07986581] 0.0965936
loss 1.17722 beta [ 0.02510442] gamma [-1.08029842] 0.0933033
loss 1.17722 beta [ 0.02510417] gamma [-1.08071661] 0.090125
loss 1.17722 beta [ 0.02510394] gamma [-1.08111942] 0.087055
loss 1.17722 beta [ 0.02510373] gamma [-1.08150911] 0.0840896
loss 1.17722 beta [ 0.02510351] gamma [-1.0818857] 0.0812252
loss 1.17722 beta [ 0.0251033] gamma [-1.08224916] 0.0784583
loss 1.17722 beta [ 0.02510309] gamma [-1.08260071] 0.0757857
loss 1.17722 beta [ 0.0251029] gamma [-1.08294034] 0.0732042
loss 1.17722 beta [ 0.02510272] gamma [-1.08326805] 0.0707106
loss 1.17722 beta [ 0.02510253] gamma [-1.08358502] 0.0683019
loss 1.17722 beta [ 0.02510234] gamma [-1.08389127] 0.0659753
loss 1.17722 beta [ 0.02510217] gamma [-1.08418679] 0.0637279
loss 1.17722 beta [ 0.025102] gamma [-1.08447278] 0.0615571
loss 1.17722 beta [ 0.02510185] gamma [-1.08474696] 0.0594602
loss 1.17722 beta [ 0

In [923]:
nb_rt = list(map(lambda x: nb_train_rated[x[1]], X_tmp_test))

In [924]:
Xt = np.column_stack((nb_rt, y_als, y_lasso))

In [925]:
loss.eval(), beta.eval(), gamma.eval()

(1.1772212,
 array([ 0.02510113], dtype=float32),
 array([-1.08598757], dtype=float32))

In [926]:
new_pred = tf.sigmoid(beta*(Xt.dot(e1) - gamma)) * y_als + (1 - tf.sigmoid(beta*(Xt.dot(e1) - gamma))) * y_lasso

In [927]:
WINDOW = 1000000

rmse_als_full = als.compute_rmse(y_als, y_tmp_test)
print('Test error', rmse_als_full)
god_rmse['als'].append(rmse_als_full)

Test error 1.18493480596


In [928]:
y_new_pred = new_pred.eval()
rmse_balse_full = als.compute_rmse(y_new_pred, y_tmp_test)
print('Test error', rmse_balse_full)
god_rmse['balse'].append(rmse_balse_full)

Test error 1.17502186735


In [929]:
rmse_lasso_full = als.compute_rmse(y_lasso, y_tmp_test)
print('Test error', rmse_lasso_full)
god_rmse['lasso'].append(rmse_lasso_full)

Test error 1.45640948077


In [930]:
for WINDOW, tag in [(1000, 'cold'), (len(nb_rt) - nb_rt.index(0), 'freeze')]:
    god_rmse['als-%s' % tag].append(als.compute_rmse(y_als[-WINDOW:], y_tmp_test[-WINDOW:]))
    y_new_pred = new_pred.eval()
    god_rmse['balse-%s' % tag].append(als.compute_rmse(y_new_pred[-WINDOW:], y_tmp_test[-WINDOW:]))
    god_rmse['lasso-%s' % tag].append(als.compute_rmse(y_lasso[-WINDOW:], y_tmp_test[-WINDOW:]))

In [935]:
for key in god_rmse:
    print(key, avgstd(god_rmse[key]))

als 1.18281 ± 0.00400
balse 1.17319 ± 0.00400
lasso 1.46214 ± 0.00500
als-cold 1.33518 ± 0.05000
balse-cold 1.25536 ± 0.03100
lasso-cold 1.33100 ± 0.01900
als-freeze 1.48283 ± 0.12200
balse-freeze 1.33063 ± 0.07200
lasso-freeze 1.35156 ± 0.03900


In [934]:
import math

def avgstd(l):  # Displays mean and variance
    n = len(l)
    mean = float(sum(l)) / n
    var = float(sum(i * i for i in l)) / n - mean * mean
    return '%.5f ± %.5f' % (round(mean, 5), round(1.96 * math.sqrt(var / n), 3))  # '%.3f ± %.3f' % 